In [1]:
import psycopg2 as ps
from wordcloud import WordCloud
import  matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import bokeh.plotting as plot
import bokeh.models as models

import re, sys
import numpy as np

sys.path.append('./TweetData')
from TweetData import keys, Preprocessing

import pandas as pd
import json
from collections import Counter
from fuzzywuzzy import fuzz, process

import geopandas as gpd

C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
places1 = places.copy()
places1.geom = places1.geom.to_crs(epsg=4326)

In [12]:
# fig = px.scatter_geo(places1,
#                     lat=places1.geom.y, lon=places1.geom.x,
#                     center = {'lat':8.9671, 'lon':-79.5359}
# )
#    zoom=3, 
#    )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig = go.Figure(
    go.Scattermapbox(
        lat=places1.geom.y, lon=places1.geom.x,
    )
)
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0},
    mapbox_style="open-street-map", mapbox_zoom=11,
    mapbox_center={'lat':8.99, 'lon':-79.5359},
    title='Mapa de lugares'
    # geo = dict(
    #     lonaxis = dict(
    #         showgrid = True,
    #         gridwidth = 0.5,
    #         range= [ -79.8582, -79.1606 ],
    #         dtick = 5
    #     ),
    #     lataxis = dict (
    #         showgrid = True,
    #         gridwidth = 0.5,
    #         range= [ 8.8023, 9.1984 ],
    #         dtick = 5
    #     )
    # ), title='US Precipitation 06-30-2015<br>Source: <a href="http://water.weather.gov/precip/">NOAA</a>',
)

fig.show()

In [2]:
## Clean text WordCloud
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
q = '''SELECT TWEET_ID, TWEET_TEXT FROM TWEETS ORDER BY TWEET_CREATED DESC '''
cursor.execute(q)
tweets = cursor.fetchall()
cursor.close()
conn.close()

tweets = pd.DataFrame(tweets, columns=['id', 'text'])

In [3]:
## Clean text WordCloud
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
# cursor = conn.cursor()
q = '''SELECT OSM_ID, NAME, ST_CENTROID(way) as geom FROM PLACES '''
# cursor.execute(q)
# places = cursor.fetchall()
places = gpd.GeoDataFrame.from_postgis(q, conn, geom_col='geom')
# cursor.close()
# conn.close()
# places = pd.DataFrame(places, columns=['id', 'name', 'geom'])

In [4]:
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
q1 = '''SELECT GID, NOMBRE, GEOM FROM carreteras WHERE NOMBRE IS NOT NULL '''
# cursor.execute(q)
# places = cursor.fetchall()
roads = gpd.GeoDataFrame.from_postgis(q1, conn, geom_col='geom')
# cursor.close()
# conn.close()
# places = pd.DataFrame(places, columns=['id', 'name', 'geom'])

In [ ]:
places1.loc[:, 'x'] = places1.apply(lambda row: getPointCoords(row, 'geom', 'x'), axis=1)
places1.loc[:, 'y'] = places1.apply(lambda row: getPointCoords(row, 'geom', 'y'), axis=1)

In [ ]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [ ]:
places = places[places.name.str.len()>5]

In [ ]:
extract = Preprocessing.clean_text(tweets, 'text')
# tweets.clean.str.findall('([A-Z]\S+)')
# re.findall('([A-Z]\S+)', tweets)
extract['possible_place'] = extract.clean.str.findall('([A-Z]\S+)')

In [ ]:
extract = extract[extract.clean.str.contains('accident.*|cho[qc].*|colision.*|vuelco|volcó|vuelca|atropell.*|tran[qc].*|embotella.*|trafico|huelga|protest.*|trabajo.*|cierr.*|detenid.*|obstacul.*|incendi.*|inunda.*|lluvia')]
extract = extract[extract.clean.str.len()>30]
extract = extract[extract.possible_place.notnull()]

In [ ]:
test = 'tranque pesado en la Via Brasil hacia el interior'
matches1 = [fuzz.token_sort_ratio(place, test) for place in places.name]
# sum(matches1)
matches2 = [fuzz.token_set_ratio(place, test) for place in places.name]
# max(matches2)

In [ ]:
process.extract(test, places, scorer=fuzz.token_set_ratio)

In [ ]:
extract['contains'] = extract['clean'].apply(lambda x: np.array([]))
# extract['contains'] = extract['contains'].astype(object)
extract['possible'] = extract['clean'].apply(lambda x: np.array([]))
# extract['possible'] = extract['possible'].astype(object)

In [ ]:
extract.drop(columns=['contains', 'possible'], inplace=True)

In [ ]:
extract2 = extract.copy()

In [ ]:
def get_road(text, roads):
    choice = process.extractOne(text, roads.nombre, scorer=fuzz.token_set_ratio)
    if choice[1]>=80:
        # Returns the gid
        return str(roads.iloc[roads.index==choice[2], 0].iloc[0])
    else: return ''

In [ ]:
def get_place(text, places):
    choice = process.extractOne(text, places.name, scorer=fuzz.token_set_ratio)
    if choice[1]>=80:
        # Returns the osm_id
        return str(places.iloc[places.index==choice[2], 0].iloc[0])
    else: return ''

In [ ]:
places.head(1)

In [ ]:
extract2 = extract[100:500]

In [ ]:
extract2.loc[:, 'road'] = extract2.apply(lambda row: get_road(row.clean, roads), axis=1)

In [ ]:
extract2.loc[:, 'place'] = extract2.apply(lambda row: get_place(row.clean, places), axis=1)

In [ ]:
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
ins = '''INSERT INTO TWEETS_PLACES(TWEET_ID, ROAD_GID, PLACE_ID)
    VALUES (%s, %s, %s) '''
for index, row in extract2.iterrows():
    cursor.execute(ins, 
        (row['id'], None if row['road']=='' else row['road'], 
            None if row['place']=='' else row['place']))
    conn.commit()
cursor.close()
conn.close()

In [ ]:
extract2

In [ ]:
extract1 = extract.copy()

In [ ]:
for index, tweet in extract[:100].iterrows():
    intweet = []; possible = []
    for index_place, place in places.iterrows():
        if len(place['name'])>5:
            if place['name'] in tweet['clean']:
                intweet.append(place['osm_id'])
                # extract.iloc[index, 'contains'] = extract.iloc[index, 'contains'].append
            elif (fuzz.token_set_ratio(place['name'], tweet['clean'])==100 and fuzz.token_sort_ratio(place['name'], tweet['clean'])>50) or (fuzz.token_set_ratio(place['name'], tweet['clean'])>90):
                possible.append(place['osm_id'])
                # print('\n', tweet, ' - possible - ', place)
            #     if fuzz.token_sort_ratio(place, tweet)>50:
            #         possible.append(place.name)
            #         # print('\n\t', tweet, ' - possible %i - ' % fuzz.token_sort_ratio(place, tweet), place)
            # elif fuzz.token_set_ratio(place, tweet)>90:
            #     possible.append(place.name)
                # print('\n', tweet, '  - similar %i to -  ' % fuzz.token_set_ratio(place, tweet), place)
    if len(intweet)>0:
        extract.iloc[index, 4] = np.array(intweet)
    if len(possible)>0:
        extract.iloc[index, 5] = np.array(possible)

In [ ]:
extract[extract.possible>0]